In [1]:
import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.14f}'.format)

In [3]:
def traiter_jsons(chemin_dossier):
    fichiers_json = [f for f in os.listdir(chemin_dossier) if f.endswith('.json')]

    def flatten(data, parent_key='', sep='_'):
        items = {}
        for k, v in data.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.update(flatten(v, new_key, sep=sep))
            elif isinstance(v, list):
                for index, element in enumerate(v):
                    if isinstance(element, dict):
                        items.update(flatten(element, f"{new_key}{sep}{index}", sep=sep))
                    else:
                        items[f"{new_key}{sep}{index}"] = element
            else:
                items[new_key] = v
        return items

    def traiter_fichier(fichier):
        chemin_fichier = os.path.join(chemin_dossier, fichier)
        try:
            with open(chemin_fichier, 'r', encoding='utf-8') as f:
                donnees = json.load(f)
                identifiant = os.path.splitext(fichier)[0]
                return [flatten({**item, 'identifiant': identifiant}) for item in donnees]
        except json.JSONDecodeError as e:
            print(f"Erreur lors de la lecture du fichier {chemin_fichier}: {str(e)}")
        except Exception as e:
            print(f"Une erreur inattendue est survenue: {str(e)}")
        return []

    resultats = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(traiter_fichier, fichier) for fichier in fichiers_json]
        for future in tqdm(as_completed(futures), total=len(futures)):
            resultats.append(future.result())

    donnees_flattened = [item for sublist in resultats for item in sublist]
    df_total = pd.DataFrame(donnees_flattened)

    return df_total

chemin_dossier = "/Users/famillediene/Downloads/Solana Wallet"
# Exécuter la fonction modifiée et afficher les premières lignes du DataFrame résultant
df_result = traiter_jsons(chemin_dossier)
df_result.head()  # Afficher les premières lignes du DataFrame


100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 12985.46it/s]


,transactionHash,data_0_instructionIndex,data_0_innerInstructionIndex,data_0_action,data_0_status,data_0_source,data_0_sourceAssociation,data_0_destination,data_0_destinationAssociation,data_0_token,data_0_amount,data_0_timestamp,data_1_instructionIndex,data_1_innerInstructionIndex,data_1_action,data_1_status,data_1_source,data_1_sourceAssociation,data_1_destination,data_1_destinationAssociation,data_1_token,data_1_amount,data_1_timestamp,data_2_instructionIndex,data_2_innerInstructionIndex,data_2_action,data_2_status,data_2_source,data_2_sourceAssociation,data_2_destination,data_2_destinationAssociation,data_2_token,data_2_amount,data_2_timestamp,data_3_instructionIndex,data_3_innerInstructionIndex,data_3_action,data_3_status,data_3_source,data_3_sourceAssociation,data_3_destination,data_3_destinationAssociation,data_3_token,data_3_amount,data_3_timestamp,identifiant,data_4_instructionIndex,data_4_innerInstructionIndex,data_4_action,data_4_status,data_4_source,data_4_sourceAssociation,data_4_destination,data_4_destinationAssociation,data_4_token,data_4_amount,data_4_timestamp,data_5_instructionIndex,data_5_innerInstructionIndex,data_5_action,data_5_status,data_5_source,data_5_sourceAssociation,data_5_destination,data_5_destinationAssociation,data_5_token,data_5_amount,data_5_timestamp,data_6_instructionIndex,data_6_innerInstructionIndex,data_6_action,data_6_status,data_6_source,data_6_sourceAssociation,data_6_destination,data_6_destinationAssociation,data_6_token,data_6_amount,data_6_timestamp,data_7_instructionIndex,data_7_innerInstructionIndex,data_7_action,data_7_status,data_7_source,data_7_sourceAssociation,data_7_destination,data_7_destinationAssociation,data_7_token,data_7_amount,data_7_timestamp,data_8_instructionIndex,data_8_innerInstructionIndex,data_8_action,data_8_status,data_8_source,data_8_sourceAssociation,data_8_destination,data_8_destinationAssociation,data_8_token,data_8_amount,data_8_timestamp,data_9_instructionIndex,data_9_innerInstructionIndex,data_9_action,data_9_status,data_9_source,data_9_sourceAssociation,data_9_destination,data_9_destinationAssociation,data_9_token,data_9_amount,data_9_timestamp,data_10_instructionIndex,data_10_innerInstructionIndex,data_10_action,data_10_status,data_10_source,data_10_sourceAssociation,data_10_destination,data_10_destinationAssociation,data_10_token,data_10_amount,data_10_timestamp,data_11_instructionIndex,data_11_innerInstructionIndex,data_11_action,data_11_status,data_11_source,data_11_sourceAssociation,data_11_destination,data_11_destinationAssociation,data_11_token,data_11_amount,data_11_timestamp,data_12_instructionIndex,data_12_innerInstructionIndex,data_12_action,data_12_status,data_12_source,data_12_sourceAssociation,data_12_destination,data_12_destinationAssociation,data_12_token,data_12_amount,data_12_timestamp,data_13_instructionIndex,data_13_innerInstructionIndex,data_13_action,data_13_status,data_13_source,data_13_sourceAssociation,data_13_destination,data_13_destinationAssociation,data_13_token,data_13_amount,data_13_timestamp,data_14_instructionIndex,data_14_innerInstructionIndex,data_14_action,data_14_status,data_14_source,data_14_sourceAssociation,data_14_destination,data_14_destinationAssociation,data_14_token,data_14_amount,data_14_timestamp,data_15_instructionIndex,data_15_innerInstructionIndex,data_15_action,data_15_status,data_15_source,data_15_sourceAssociation,data_15_destination,data_15_destinationAssociation,data_15_token,data_15_amount,data_15_timestamp,data_16_instructionIndex,data_16_innerInstructionIndex,data_16_action,data_16_status,data_16_source,data_16_sourceAssociation,data_16_destination,data_16_destinationAssociation,data_16_token,data_16_amount,data_16_timestamp,data_17_instructionIndex,data_17_innerInstructionIndex,data_17_action,data_17_status,data_17_source,data_17_sourceAssociation,data_17_destination,data_17_destinationAssociation,data_17_token,data_17_amount,data_17_timestamp,data_18_instructionIndex,data_18_innerInstruction

In [4]:
df_result=df_result[df_result['data_1_timestamp'].notna()]

In [5]:
df_result

,transactionHash,data_0_instructionIndex,data_0_innerInstructionIndex,data_0_action,data_0_status,data_0_source,data_0_sourceAssociation,data_0_destination,data_0_destinationAssociation,data_0_token,data_0_amount,data_0_timestamp,data_1_instructionIndex,data_1_innerInstructionIndex,data_1_action,data_1_status,data_1_source,data_1_sourceAssociation,data_1_destination,data_1_destinationAssociation,data_1_token,data_1_amount,data_1_timestamp,data_2_instructionIndex,data_2_innerInstructionIndex,data_2_action,data_2_status,data_2_source,data_2_sourceAssociation,data_2_destination,data_2_destinationAssociation,data_2_token,data_2_amount,data_2_timestamp,data_3_instructionIndex,data_3_innerInstructionIndex,data_3_action,data_3_status,data_3_source,data_3_sourceAssociation,data_3_destination,data_3_destinationAssociation,data_3_token,data_3_amount,data_3_timestamp,identifiant,data_4_instructionIndex,data_4_innerInstructionIndex,data_4_action,data_4_status,data_4_source,data_4_sourceAssociation,data_4_destination,data_4_destinationAssociation,data_4_token,data_4_amount,data_4_timestamp,data_5_instructionIndex,data_5_innerInstructionIndex,data_5_action,data_5_status,data_5_source,data_5_sourceAssociation,data_5_destination,data_5_destinationAssociation,data_5_token,data_5_amount,data_5_timestamp,data_6_instructionIndex,data_6_innerInstructionIndex,data_6_action,data_6_status,data_6_source,data_6_sourceAssociation,data_6_destination,data_6_destinationAssociation,data_6_token,data_6_amount,data_6_timestamp,data_7_instructionIndex,data_7_innerInstructionIndex,data_7_action,data_7_status,data_7_source,data_7_sourceAssociation,data_7_destination,data_7_destinationAssociation,data_7_token,data_7_amount,data_7_timestamp,data_8_instructionIndex,data_8_innerInstructionIndex,data_8_action,data_8_status,data_8_source,data_8_sourceAssociation,data_8_destination,data_8_destinationAssociation,data_8_token,data_8_amount,data_8_timestamp,data_9_instructionIndex,data_9_innerInstructionIndex,data_9_action,data_9_status,data_9_source,data_9_sourceAssociation,data_9_destination,data_9_destinationAssociation,data_9_token,data_9_amount,data_9_timestamp,data_10_instructionIndex,data_10_innerInstructionIndex,data_10_action,data_10_status,data_10_source,data_10_sourceAssociation,data_10_destination,data_10_destinationAssociation,data_10_token,data_10_amount,data_10_timestamp,data_11_instructionIndex,data_11_innerInstructionIndex,data_11_action,data_11_status,data_11_source,data_11_sourceAssociation,data_11_destination,data_11_destinationAssociation,data_11_token,data_11_amount,data_11_timestamp,data_12_instructionIndex,data_12_innerInstructionIndex,data_12_action,data_12_status,data_12_source,data_12_sourceAssociation,data_12_destination,data_12_destinationAssociation,data_12_token,data_12_amount,data_12_timestamp,data_13_instructionIndex,data_13_innerInstructionIndex,data_13_action,data_13_status,data_13_source,data_13_sourceAssociation,data_13_destination,data_13_destinationAssociation,data_13_token,data_13_amount,data_13_timestamp,data_14_instructionIndex,data_14_innerInstructionIndex,data_14_action,data_14_status,data_14_source,data_14_sourceAssociation,data_14_destination,data_14_destinationAssociation,data_14_token,data_14_amount,data_14_timestamp,data_15_instructionIndex,data_15_innerInstructionIndex,data_15_action,data_15_status,data_15_source,data_15_sourceAssociation,data_15_destination,data_15_destinationAssociation,data_15_token,data_15_amount,data_15_timestamp,data_16_instructionIndex,data_16_innerInstructionIndex,data_16_action,data_16_status,data_16_source,data_16_sourceAssociation,data_16_destination,data_16_destinationAssociation,data_16_token,data_16_amount,data_16_timestamp,data_17_instructionIndex,data_17_innerInstructionIndex,data_17_action,data_17_status,data_17_source,data_17_sourceAssociation,data_17_destination,data_17_destinationAssociation,data_17_token,data_17_amount,data_17_timestamp,data_18_instructionIndex,data_18_innerInstruction

In [6]:
df_result['identifiant'].value_counts()

response_1706711488156    58
response_1706711684049    51
response_1706711726210    17
Name: identifiant, dtype: int64

In [7]:
df_result2=df_result[df_result['identifiant'].str.contains('response_1706711726210')]

In [8]:
def convert_timestamp_to_date(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%d-%m-%Y %H:%M:%S')

if 'data_1_timestamp' in df_result.columns:
    df_result2['data_1_timestamp'] = df_result2['data_1_timestamp'].apply(convert_timestamp_to_date)
    
df_result2

/var/folders/43/p13r_kws67708f3v4c0fzfp80000gn/T/ipykernel_46851/1242525506.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result2['data_1_timestamp'] = df_result2['data_1_timestamp'].apply(convert_timestamp_to_date)


,transactionHash,data_0_instructionIndex,data_0_innerInstructionIndex,data_0_action,data_0_status,data_0_source,data_0_sourceAssociation,data_0_destination,data_0_destinationAssociation,data_0_token,data_0_amount,data_0_timestamp,data_1_instructionIndex,data_1_innerInstructionIndex,data_1_action,data_1_status,data_1_source,data_1_sourceAssociation,data_1_destination,data_1_destinationAssociation,data_1_token,data_1_amount,data_1_timestamp,data_2_instructionIndex,data_2_innerInstructionIndex,data_2_action,data_2_status,data_2_source,data_2_sourceAssociation,data_2_destination,data_2_destinationAssociation,data_2_token,data_2_amount,data_2_timestamp,data_3_instructionIndex,data_3_innerInstructionIndex,data_3_action,data_3_status,data_3_source,data_3_sourceAssociation,data_3_destination,data_3_destinationAssociation,data_3_token,data_3_amount,data_3_timestamp,identifiant,data_4_instructionIndex,data_4_innerInstructionIndex,data_4_action,data_4_status,data_4_source,data_4_sourceAssociation,data_4_destination,data_4_destinationAssociation,data_4_token,data_4_amount,data_4_timestamp,data_5_instructionIndex,data_5_innerInstructionIndex,data_5_action,data_5_status,data_5_source,data_5_sourceAssociation,data_5_destination,data_5_destinationAssociation,data_5_token,data_5_amount,data_5_timestamp,data_6_instructionIndex,data_6_innerInstructionIndex,data_6_action,data_6_status,data_6_source,data_6_sourceAssociation,data_6_destination,data_6_destinationAssociation,data_6_token,data_6_amount,data_6_timestamp,data_7_instructionIndex,data_7_innerInstructionIndex,data_7_action,data_7_status,data_7_source,data_7_sourceAssociation,data_7_destination,data_7_destinationAssociation,data_7_token,data_7_amount,data_7_timestamp,data_8_instructionIndex,data_8_innerInstructionIndex,data_8_action,data_8_status,data_8_source,data_8_sourceAssociation,data_8_destination,data_8_destinationAssociation,data_8_token,data_8_amount,data_8_timestamp,data_9_instructionIndex,data_9_innerInstructionIndex,data_9_action,data_9_status,data_9_source,data_9_sourceAssociation,data_9_destination,data_9_destinationAssociation,data_9_token,data_9_amount,data_9_timestamp,data_10_instructionIndex,data_10_innerInstructionIndex,data_10_action,data_10_status,data_10_source,data_10_sourceAssociation,data_10_destination,data_10_destinationAssociation,data_10_token,data_10_amount,data_10_timestamp,data_11_instructionIndex,data_11_innerInstructionIndex,data_11_action,data_11_status,data_11_source,data_11_sourceAssociation,data_11_destination,data_11_destinationAssociation,data_11_token,data_11_amount,data_11_timestamp,data_12_instructionIndex,data_12_innerInstructionIndex,data_12_action,data_12_status,data_12_source,data_12_sourceAssociation,data_12_destination,data_12_destinationAssociation,data_12_token,data_12_amount,data_12_timestamp,data_13_instructionIndex,data_13_innerInstructionIndex,data_13_action,data_13_status,data_13_source,data_13_sourceAssociation,data_13_destination,data_13_destinationAssociation,data_13_token,data_13_amount,data_13_timestamp,data_14_instructionIndex,data_14_innerInstructionIndex,data_14_action,data_14_status,data_14_source,data_14_sourceAssociation,data_14_destination,data_14_destinationAssociation,data_14_token,data_14_amount,data_14_timestamp,data_15_instructionIndex,data_15_innerInstructionIndex,data_15_action,data_15_status,data_15_source,data_15_sourceAssociation,data_15_destination,data_15_destinationAssociation,data_15_token,data_15_amount,data_15_timestamp,data_16_instructionIndex,data_16_innerInstructionIndex,data_16_action,data_16_status,data_16_source,data_16_sourceAssociation,data_16_destination,data_16_destinationAssociation,data_16_token,data_16_amount,data_16_timestamp,data_17_instructionIndex,data_17_innerInstructionIndex,data_17_action,data_17_status,data_17_source,data_17_sourceAssociation,data_17_destination,data_17_destinationAssociation,data_17_token,data_17_amount,data_17_timestamp,data_18_instructionIndex,data_18_innerInstruction

In [9]:
df_result2['data_1_timestamp'] = pd.to_datetime(df_result2['data_1_timestamp'], format='%d-%m-%Y %H:%M:%S')

df_result2 = df_result2.sort_values(by='data_1_timestamp')
df_result2

/var/folders/43/p13r_kws67708f3v4c0fzfp80000gn/T/ipykernel_46851/2246674634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result2['data_1_timestamp'] = pd.to_datetime(df_result2['data_1_timestamp'], format='%d-%m-%Y %H:%M:%S')


,transactionHash,data_0_instructionIndex,data_0_innerInstructionIndex,data_0_action,data_0_status,data_0_source,data_0_sourceAssociation,data_0_destination,data_0_destinationAssociation,data_0_token,data_0_amount,data_0_timestamp,data_1_instructionIndex,data_1_innerInstructionIndex,data_1_action,data_1_status,data_1_source,data_1_sourceAssociation,data_1_destination,data_1_destinationAssociation,data_1_token,data_1_amount,data_1_timestamp,data_2_instructionIndex,data_2_innerInstructionIndex,data_2_action,data_2_status,data_2_source,data_2_sourceAssociation,data_2_destination,data_2_destinationAssociation,data_2_token,data_2_amount,data_2_timestamp,data_3_instructionIndex,data_3_innerInstructionIndex,data_3_action,data_3_status,data_3_source,data_3_sourceAssociation,data_3_destination,data_3_destinationAssociation,data_3_token,data_3_amount,data_3_timestamp,identifiant,data_4_instructionIndex,data_4_innerInstructionIndex,data_4_action,data_4_status,data_4_source,data_4_sourceAssociation,data_4_destination,data_4_destinationAssociation,data_4_token,data_4_amount,data_4_timestamp,data_5_instructionIndex,data_5_innerInstructionIndex,data_5_action,data_5_status,data_5_source,data_5_sourceAssociation,data_5_destination,data_5_destinationAssociation,data_5_token,data_5_amount,data_5_timestamp,data_6_instructionIndex,data_6_innerInstructionIndex,data_6_action,data_6_status,data_6_source,data_6_sourceAssociation,data_6_destination,data_6_destinationAssociation,data_6_token,data_6_amount,data_6_timestamp,data_7_instructionIndex,data_7_innerInstructionIndex,data_7_action,data_7_status,data_7_source,data_7_sourceAssociation,data_7_destination,data_7_destinationAssociation,data_7_token,data_7_amount,data_7_timestamp,data_8_instructionIndex,data_8_innerInstructionIndex,data_8_action,data_8_status,data_8_source,data_8_sourceAssociation,data_8_destination,data_8_destinationAssociation,data_8_token,data_8_amount,data_8_timestamp,data_9_instructionIndex,data_9_innerInstructionIndex,data_9_action,data_9_status,data_9_source,data_9_sourceAssociation,data_9_destination,data_9_destinationAssociation,data_9_token,data_9_amount,data_9_timestamp,data_10_instructionIndex,data_10_innerInstructionIndex,data_10_action,data_10_status,data_10_source,data_10_sourceAssociation,data_10_destination,data_10_destinationAssociation,data_10_token,data_10_amount,data_10_timestamp,data_11_instructionIndex,data_11_innerInstructionIndex,data_11_action,data_11_status,data_11_source,data_11_sourceAssociation,data_11_destination,data_11_destinationAssociation,data_11_token,data_11_amount,data_11_timestamp,data_12_instructionIndex,data_12_innerInstructionIndex,data_12_action,data_12_status,data_12_source,data_12_sourceAssociation,data_12_destination,data_12_destinationAssociation,data_12_token,data_12_amount,data_12_timestamp,data_13_instructionIndex,data_13_innerInstructionIndex,data_13_action,data_13_status,data_13_source,data_13_sourceAssociation,data_13_destination,data_13_destinationAssociation,data_13_token,data_13_amount,data_13_timestamp,data_14_instructionIndex,data_14_innerInstructionIndex,data_14_action,data_14_status,data_14_source,data_14_sourceAssociation,data_14_destination,data_14_destinationAssociation,data_14_token,data_14_amount,data_14_timestamp,data_15_instructionIndex,data_15_innerInstructionIndex,data_15_action,data_15_status,data_15_source,data_15_sourceAssociation,data_15_destination,data_15_destinationAssociation,data_15_token,data_15_amount,data_15_timestamp,data_16_instructionIndex,data_16_innerInstructionIndex,data_16_action,data_16_status,data_16_source,data_16_sourceAssociation,data_16_destination,data_16_destinationAssociation,data_16_token,data_16_amount,data_16_timestamp,data_17_instructionIndex,data_17_innerInstructionIndex,data_17_action,data_17_status,data_17_source,data_17_sourceAssociation,data_17_destination,data_17_destinationAssociation,data_17_token,data_17_amount,data_17_timestamp,data_18_instructionIndex,data_18_innerInstruction

In [7]:
file_path="/Users/famillediene/Downloads/TAG_Solana_0.csv"

In [8]:
df_label=pd.read_csv(file_path, delimiter=';', quotechar='"', escapechar='\\')
df_label=df_label[df_label['Smart Contract'].notna()]
df_label

,sc-fmvJgu,Nom Projet,dapp-name-link-comp href,Nom token,MarketCap,Catégorie 1,Categorie 2,Smart Contract,Token,SC
0,NaN,_PixusMetaverse,https://dappradar.com/nft-collection/pixusmetaverse,NaN,NaN,NFT,Collectible,6scKBny6gFuhxaebpTGxaxPz23ej25roPTMgmCPdf8EF,NaN,NaN
1,NaN,1inch,https://dappradar.com/token/1inch,1inch,"$362,07M",Token DeFi,NaN,5wihEYGca7X4gSe97C5mVcqNsfxBzhdTwpv72HKs25US,NaN,NaN
2,NaN,7Solana,https://dappradar.com/dapp/7solana,NaN,NaN,DeFi,Stacking,5QjYzMZThuwXAW16tBDUuqP2KYf3mwKAPL1FZmTMdtCu,NaN,X
3,NaN,Aave,https://dappradar.com/token/aave,aave,"$1,41B",DeFi,Gouvernance,dK83wTVypEpa1pqiBbHY3MNuUnT3ADUZM4wk9VZXZEc,NaN,NaN
4,NaN,Abstratica,https://dappradar.com/nft-collection/abstratica,NaN,NaN,NFT,NaN,absF5t1MP7dXFhbgUgaTi7ffJM2WhBg1cmrY5s7h3By,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
360,NaN,USDK,https://dappradar.com/token/usdk,usdk,"$441,49k",Stablecoin,NaN,43m2ewFV5nDepieFjT9EmAQnc1HRtAF247RBpLGFem5F,NaN,NaN
365,NaN,WETH,https://dappradar.com/token/weth,weth,-,Wrapped Asset,Bridged (allBridge),AaAEw2VCw1XzgvKB8Rj2DyK2ZVau9fbt2bE8hZFWsMyE,NaN,NaN
366,NaN,WETH,NaN,weth,NaN,Wrapped Asset,Bridged (Wormhole),7vfCXTUXx5WJV5JADk17DUJ4ksgau7utNKj4b963voxs,NaN,NaN
367,NaN,Whales Nation,https://dappradar.com/nft-collection/whales-nation,NaN,NaN,NFT,NaN,2kLQM2N4MZmEX1VXso7AkNUMSWwrhi74M1dWiXsKMmCJ,NaN,NaN


In [9]:
df_result['data_1_token'] = df_result['data_1_token'].replace('', np.nan)
df_result

,transactionHash,data_0_instructionIndex,data_0_innerInstructionIndex,data_0_action,data_0_status,data_0_source,data_0_sourceAssociation,data_0_destination,data_0_destinationAssociation,data_0_token,data_0_amount,data_0_timestamp,identifiant,data_1_instructionIndex,data_1_innerInstructionIndex,data_1_action,data_1_status,data_1_source,data_1_sourceAssociation,data_1_destination,data_1_destinationAssociation,data_1_token,data_1_amount,data_1_timestamp,data_2_instructionIndex,data_2_innerInstructionIndex,data_2_action,data_2_status,data_2_source,data_2_sourceAssociation,data_2_destination,data_2_destinationAssociation,data_2_token,data_2_amount,data_2_timestamp,data_3_instructionIndex,data_3_innerInstructionIndex,data_3_action,data_3_status,data_3_source,data_3_sourceAssociation,data_3_destination,data_3_destinationAssociation,data_3_token,data_3_amount,data_3_timestamp,data_4_instructionIndex,data_4_innerInstructionIndex,data_4_action,data_4_status,data_4_source,data_4_sourceAssociation,data_4_destination,data_4_destinationAssociation,data_4_token,data_4_amount,data_4_timestamp,data_5_instructionIndex,data_5_innerInstructionIndex,data_5_action,data_5_status,data_5_source,data_5_sourceAssociation,data_5_destination,data_5_destinationAssociation,data_5_token,data_5_amount,data_5_timestamp,data_6_instructionIndex,data_6_innerInstructionIndex,data_6_action,data_6_status,data_6_source,data_6_sourceAssociation,data_6_destination,data_6_destinationAssociation,data_6_token,data_6_amount,data_6_timestamp,data_7_instructionIndex,data_7_innerInstructionIndex,data_7_action,data_7_status,data_7_source,data_7_sourceAssociation,data_7_destination,data_7_destinationAssociation,data_7_token,data_7_amount,data_7_timestamp,data_8_instructionIndex,data_8_innerInstructionIndex,data_8_action,data_8_status,data_8_source,data_8_sourceAssociation,data_8_destination,data_8_destinationAssociation,data_8_token,data_8_amount,data_8_timestamp,data_9_instructionIndex,data_9_innerInstructionIndex,data_9_action,data_9_status,data_9_source,data_9_sourceAssociation,data_9_destination,data_9_destinationAssociation,data_9_token,data_9_amount,data_9_timestamp,data_10_instructionIndex,data_10_innerInstructionIndex,data_10_action,data_10_status,data_10_source,data_10_sourceAssociation,data_10_destination,data_10_destinationAssociation,data_10_token,data_10_amount,data_10_timestamp,data_11_instructionIndex,data_11_innerInstructionIndex,data_11_action,data_11_status,data_11_source,data_11_sourceAssociation,data_11_destination,data_11_destinationAssociation,data_11_token,data_11_amount,data_11_timestamp,data_12_instructionIndex,data_12_innerInstructionIndex,data_12_action,data_12_status,data_12_source,data_12_sourceAssociation,data_12_destination,data_12_destinationAssociation,data_12_token,data_12_amount,data_12_timestamp,data_13_instructionIndex,data_13_innerInstructionIndex,data_13_action,data_13_status,data_13_source,data_13_sourceAssociation,data_13_destination,data_13_destinationAssociation,data_13_token,data_13_amount,data_13_timestamp,data_14_instructionIndex,data_14_innerInstructionIndex,data_14_action,data_14_status,data_14_source,data_14_sourceAssociation,data_14_destination,data_14_destinationAssociation,data_14_token,data_14_amount,data_14_timestamp,data_15_instructionIndex,data_15_innerInstructionIndex,data_15_action,data_15_status,data_15_source,data_15_sourceAssociation,data_15_destination,data_15_destinationAssociation,data_15_token,data_15_amount,data_15_timestamp,data_16_instructionIndex,data_16_innerInstructionIndex,data_16_action,data_16_status,data_16_source,data_16_sourceAssociation,data_16_destination,data_16_destinationAssociation,data_16_token,data_16_amount,data_16_timestamp,data_17_instructionIndex,data_17_innerInstructionIndex,data_17_action,data_17_status,data_17_source,data_17_sourceAssociation,data_17_destination,data_17_destinationAssociation,data_17_token,data_17_amount,data_17_timestamp,data_18_instructionIndex,data_18_innerInstruction

In [10]:
# Renommer la colonne "Smart Contract" dans df_label_sol pour qu'elle corresponde à "data_1_token"
df_label_sol_renamed = df_label.rename(columns={"Smart Contract": "token_to_use"})

# Créer une nouvelle colonne "token_to_use" dans df_solana2 qui utilise "data_1_token" si non vide, sinon "data_2_token"
df_result['token_to_use'] = np.where(df_result['data_1_token'].isna(), df_result['data_2_token'], df_result['data_1_token'])

# Jointure des deux dataframes sur la nouvelle colonne "token_to_use"
final_df = pd.merge(df_result, df_label_sol_renamed[['Nom Projet', 'Nom token','Catégorie 1','Categorie 2','token_to_use']], on='token_to_use', how='left')

final_df

,transactionHash,data_0_instructionIndex,data_0_innerInstructionIndex,data_0_action,data_0_status,data_0_source,data_0_sourceAssociation,data_0_destination,data_0_destinationAssociation,data_0_token,data_0_amount,data_0_timestamp,identifiant,data_1_instructionIndex,data_1_innerInstructionIndex,data_1_action,data_1_status,data_1_source,data_1_sourceAssociation,data_1_destination,data_1_destinationAssociation,data_1_token,data_1_amount,data_1_timestamp,data_2_instructionIndex,data_2_innerInstructionIndex,data_2_action,data_2_status,data_2_source,data_2_sourceAssociation,data_2_destination,data_2_destinationAssociation,data_2_token,data_2_amount,data_2_timestamp,data_3_instructionIndex,data_3_innerInstructionIndex,data_3_action,data_3_status,data_3_source,data_3_sourceAssociation,data_3_destination,data_3_destinationAssociation,data_3_token,data_3_amount,data_3_timestamp,data_4_instructionIndex,data_4_innerInstructionIndex,data_4_action,data_4_status,data_4_source,data_4_sourceAssociation,data_4_destination,data_4_destinationAssociation,data_4_token,data_4_amount,data_4_timestamp,data_5_instructionIndex,data_5_innerInstructionIndex,data_5_action,data_5_status,data_5_source,data_5_sourceAssociation,data_5_destination,data_5_destinationAssociation,data_5_token,data_5_amount,data_5_timestamp,data_6_instructionIndex,data_6_innerInstructionIndex,data_6_action,data_6_status,data_6_source,data_6_sourceAssociation,data_6_destination,data_6_destinationAssociation,data_6_token,data_6_amount,data_6_timestamp,data_7_instructionIndex,data_7_innerInstructionIndex,data_7_action,data_7_status,data_7_source,data_7_sourceAssociation,data_7_destination,data_7_destinationAssociation,data_7_token,data_7_amount,data_7_timestamp,data_8_instructionIndex,data_8_innerInstructionIndex,data_8_action,data_8_status,data_8_source,data_8_sourceAssociation,data_8_destination,data_8_destinationAssociation,data_8_token,data_8_amount,data_8_timestamp,data_9_instructionIndex,data_9_innerInstructionIndex,data_9_action,data_9_status,data_9_source,data_9_sourceAssociation,data_9_destination,data_9_destinationAssociation,data_9_token,data_9_amount,data_9_timestamp,data_10_instructionIndex,data_10_innerInstructionIndex,data_10_action,data_10_status,data_10_source,data_10_sourceAssociation,data_10_destination,data_10_destinationAssociation,data_10_token,data_10_amount,data_10_timestamp,data_11_instructionIndex,data_11_innerInstructionIndex,data_11_action,data_11_status,data_11_source,data_11_sourceAssociation,data_11_destination,data_11_destinationAssociation,data_11_token,data_11_amount,data_11_timestamp,data_12_instructionIndex,data_12_innerInstructionIndex,data_12_action,data_12_status,data_12_source,data_12_sourceAssociation,data_12_destination,data_12_destinationAssociation,data_12_token,data_12_amount,data_12_timestamp,data_13_instructionIndex,data_13_innerInstructionIndex,data_13_action,data_13_status,data_13_source,data_13_sourceAssociation,data_13_destination,data_13_destinationAssociation,data_13_token,data_13_amount,data_13_timestamp,data_14_instructionIndex,data_14_innerInstructionIndex,data_14_action,data_14_status,data_14_source,data_14_sourceAssociation,data_14_destination,data_14_destinationAssociation,data_14_token,data_14_amount,data_14_timestamp,data_15_instructionIndex,data_15_innerInstructionIndex,data_15_action,data_15_status,data_15_source,data_15_sourceAssociation,data_15_destination,data_15_destinationAssociation,data_15_token,data_15_amount,data_15_timestamp,data_16_instructionIndex,data_16_innerInstructionIndex,data_16_action,data_16_status,data_16_source,data_16_sourceAssociation,data_16_destination,data_16_destinationAssociation,data_16_token,data_16_amount,data_16_timestamp,data_17_instructionIndex,data_17_innerInstructionIndex,data_17_action,data_17_status,data_17_source,data_17_sourceAssociation,data_17_destination,data_17_destinationAssociation,data_17_token,data_17_amount,data_17_timestamp,data_18_instructionIndex,data_18_innerInstruction

In [11]:
df = final_df.rename(columns={
    'data_1_timestamp': 'Date',
    'identifiant': 'Wallet_Id',
    'data_1_destination' : 'Receiver',
    'data_1_source' : 'Sender',
    'data_1_action': 'Operation',
    'data_1_status': 'Status',
    'Nom token': 'Tokens_1',
    'data_1_amount': 'Nb_Tokens',
    'data_2_amount': 'Nb_Tokens2',
    'data_0_amount': 'Fees',
    'Nom Projet': 'Name',
    'Catégorie 1': 'Category_1',
    'Categorie 2': 'Category_2',
    'transactionHash':'Id',
})

In [12]:
df=df[['Date','Wallet_Id','Receiver','Sender','Operation','Status','Tokens_1','Nb_Tokens','Nb_Tokens2','Fees','Name','Category_1','Category_2','token_to_use','Id']]

df['Nb_Tokens']=df['Nb_Tokens'] / 10**6
df['Nb_Tokens2']=df['Nb_Tokens2'] / 10**6
df['Fees']=df['Fees'] / 10**9

In [13]:
df['Tokens_1'] = df['Tokens_1'].fillna('sol')

df['Nb_Tokens'] = np.where(
    df['Tokens_1'].str.contains('usdc|usdt', na=False),
    df['Nb_Tokens'] / 10**6,
    np.where(
        df['Tokens_1'].str.contains('sol', na=False),
        df['Nb_Tokens'] / 10**9,
        df['Nb_Tokens']  # default case if neither condition is met
    )
)

In [14]:
df['Nb_Tokens2']=df['Nb_Tokens2'] / 10**9

In [17]:
df['token_to_use'].unique()

array(['EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v', nan, '',
       '7yBzFHHhYfkr7dtYj4qEowmhbudRXauGBSNfwpGutK2R',
       '8rUiXmfbtf611bxmcM9u8wVdT3n5UwUCCfdS8DWMog1B',
       'CkeWuFpM72264LA4ptPpwQYPUXp11GeCHZV1ex7B7gd8',
       'GDvQr3jmje84KUspQ4k1cuAC1NURESYqbvF6VyDDaPPK',
       'So11111111111111111111111111111111111111112',
       'WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk',
       'BQH7CM79CJ33ZKUhCrCqerHWJx1FXAYD9QWB7c2xR3Kp',
       '657D8uX3bcHvpvFy28RtvWNCaoZSvwPNnyyAGBGSuGCf',
       'B5ubkHqxVy3zSqxCtQuX8SBWR7fD5ghGFKaSwENqyPoS',
       'AS2NCLijnFeM4bsJ2Z1dCi9pb95WoDWBjQ5Z7MZDQ19o',
       'HWac2jwydz6u5b9V6F8PGrg1qaMkZzEFPU7kZgt4T2xY',
       'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
       '34SvzLDGSgz4Jkr2Xhq2MxoHX8kNSZ5MagYf21NVcgxz',
       '5t39rdKbwrMk9Refms5XnN8Xba7s97SZ8QvLm1M9vmte',
       'LSTxxxnJzKDFSLr4dUkPcmCf5VyryEqzPLz5j4bpxFp',
       '25a3M4eeJoCsaB4QUr52paYcDk1vT5s7eGBmExbj9Cnf',
       'V4pnsMWC7PpAcAZ4DZhsdzLttFcNwxTL8ZMbdiusSzE',
      

In [21]:
df['Sender'].value_counts()

HeFPmhqGMZbvCKLfuwUyj8YSAFYf17DBzLf2bJM9GA2k    26
9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru    22
FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7    17
APHroyxXksXMLAVgpEKVvspiupBUshaPYxsoJjF2SzzV     9
3T1fQuahAAwNGLoVdbLLviJyMVetsLt1KqShWSnFEtS4     9
9feoKCvkwWse1ynubhkoQWY9SPdnAqMbCVF7si7aJevW     2
2ojv9BAiHUrvsm9gxDe7fJSzbNZSJcxZvf8dqmWGHG8S     2
pyEk8dXMtdq5pbYENjmzHqKGgZEZE3mijn7VymcRfFa      2
Py6vwpUPkFrkRPsRcM93otZ2dRipKJnWLUwK8dSmjZe      2
DQ5JWbJyWdJeyBxZuuyu36sUBud6L6wo3aN1QC1bRmsR     2
88z4NBtRbmXLRehUtVzuHfi5V1WKrFcoKRYTZjN9QdE4     2
FBQHeifBur7UDEUkBV9v16TAYfhDTnpjbwV8GTdp5jMq     2
46DhkiAyZMxarDJA11KivzJ3dDhWYSnoL4k9jcQzU3go     2
AKumNwdrb42CRtnWCNmCsQ2fketLHb7zrLFhECVsA4cT     2
DbF7cjsq6aBifX9ogr2JMAqfbHVhXvNJSzm7nXc3SMq1     1
FMewbD7tPn9ukcBp5UsBD1zpqF51sNYv2fjEudbB1M5d     1
61X8MNJW3gQP6E75p3Mf97LKN5QjiES3wqz7pJ7ue76e     1
PPpMGHom4FvQP2X3JRJg4Qowt9R19jx7DkHeNFUtf6t      1
dropjakoD4FsXK24Cd6SKDikcQrB4acr9uXeVFgfe4b      1
6NJgtk9c1RDcezbMCaJ8Pg88htJjseQ

In [ ]:
EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v usdc
So11111111111111111111111111111111111111112 wsol
WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk wen 

In [16]:
df[df['Category_1'].notna()]

,Date,Wallet_Id,Receiver,Sender,Operation,Status,Tokens_1,Nb_Tokens,Nb_Tokens2,Fees,Name,Category_1,Category_2,token_to_use,Id
0,18-12-2023 12:57:21,response_1706711726210,7VsvGE3aWciXxq9wRdX89AqkNMfQhxv2JPtjuquzk3QN,APHroyxXksXMLAVgpEKVvspiupBUshaPYxsoJjF2SzzV,createAccount,Successful,usdc,1.83744000000000,0.03000000000000,25000,USD Coin,Stablecoin,NaN,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,2SzxEMihjRSzKGqC8yaAYAsvTt7DwvtKzANGGo483wYGC1aF3jsDNiWe99dvYANMrK3bX5Bzu2emWaSBQmmpKBym
5,18-12-2023 12:49:04,response_1706711726210,82dGS7Jt4Km8ZgwZVRsJ2V6vPXEhVdgDaMP7cqPGG1TW,APHroyxXksXMLAVgpEKVvspiupBUshaPYxsoJjF2SzzV,transfer,Successful,usdc,30.00000000000000,NaN,15000,USD Coin,Stablecoin,NaN,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,3XgdFLr8XinhNjwdcgRkXcWWQugVtT2F3AbYadf6pksdgg872sxwW6SfnPeuSyLbjEDGbUqPVoFhz1cMThgByukv
29,09-12-2023 11:03:31,response_1706711488156,2q4E7vEinvkk4oQwfBA5ySYxsXm9VZUj43X8jgYzUZN6,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,createAccount,Successful,msol,0.00203928000000,1.00000000000000,15000,Marinade staked SOL,DeFi,Stacking,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2uCD8EnkPEu2BWKBoxfrVMVxrnLRbXJ8eeu1GSx6WHjGFkTrtYr6X8py1x2XWnnqeGGkLkVxdN9uGWvBu23EESvc
37,08-12-2023 22:48:53,response_1706711488156,3uxNepDbmkDNq6JhRja5Z8QwbTrfmkKP8AKZV5chYDGG,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,transfer,Successful,usdc,9.00000000000000,NaN,46500,USD Coin,Stablecoin,NaN,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,3etdR6TNFWtqpUcKV1hcgiPLuzyBJ8HGR5cppxZsNxy7RWtTuseKmJh4m6zAgFQ4u6AA6sW1pqytA3ZSGSTvNgNm
39,08-12-2023 23:29:35,response_1706711488156,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,6YxGd65JbXzgFGWjE44jsyVeCnZp7Bb1wfL9jDia1n8w,transfer,Successful,msol,0.15000000000000,NaN,25000,Marinade staked SOL,DeFi,Stacking,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,3oB8ReudwTH6EYquPpjWtyXmKESL1M94D5CVPaG4cUk7Egzau3tQGbsaxtYCrFn3ZSES1REA7uRZHhdYLGRPRWRe
40,08-12-2023 23:19:46,response_1706711488156,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,3uxNepDbmkDNq6JhRja5Z8QwbTrfmkKP8AKZV5chYDGG,transfer,Successful,usdc,1.00000000000000,NaN,92000,USD Coin,Stablecoin,NaN,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,3oykGtdAkmRdYGQDS2mTmETDVNPoYBMFgmh2SKPynpDoCZFb21bbMqFdjMQKbL96UJmrK6Js1VjikMpEPpWJbJVB
70,15-04-2022 16:42:58,response_1706711488156,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,EyaSjUtSgo9aRD1f8LWXwdvkpDTmXAW54yoSHZRF14WL,transfer,Successful,msol,8.63820892100000,9.00000000000000,5000,Marinade staked SOL,DeFi,Stacking,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2oRupYu7ngnLBnHUxXR8zZt3vKKLhwT2YznPD7N6s2UbqZf5BZ4RchSnj9R5FMJeh1D5sJD2ihxjVGmNcgSoUNU3
74,15-04-2022 16:41:28,response_1706711488156,4bLaeJv3tGAVWBHK4ptwm9oFmyyooP5Ubw85oDv8zmJ1,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,transfer,Successful,msol,0.00203928000000,0.95980099100000,5000,Marinade staked SOL,DeFi,Stacking,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,qWF1QCbXYVdA4w38mUWcCy4yxq84ondfvUyVKiDAUTjAchU35yf9HBvPXMSoX2eervtXmJCWxVAzTRrTStZKDZj
85,09-12-2023 11:03:31,response_1706711684049,2q4E7vEinvkk4oQwfBA5ySYxsXm9VZUj43X8jgYzUZN6,9FSWLLxQV2JoVbPoeAM7AZyBb35MpepfFDVNhTnARHru,createAccount,Successful,msol,0.00203928000000,1.00000000000000,15000,Marinade staked SOL,DeFi,Stacking,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2uCD8EnkPEu2BWKBoxfrVMVxrnLRbXJ8eeu1GSx6WHjGFkTrtYr6X8py1x2XWnnqeGGkLkVxdN9uGWvBu23EESvc
92,18-12-2023 12:56:23,response_1706711684049,APHroyxXksXMLAVgpEKVvspiupBUshaPYxsoJjF2SzzV,HeFPmhqGMZbvCKLfuwUyj8YSAFYf17DBzLf2bJM9GA2k,transferChecked,Successful,usdc,30.00000000000000,NaN,15000,USD Coin,Stablecoin,NaN,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,3nvRHRuqa8mNFDv2bVimTdVvaewfVf88Yp2LDrRKGGMeEySsmS6f7wdejnpdNqDmUWYbDWe8GMn6uB3s1F36CTM9


# Spécifiez le chemin du fichier CSV dans lequel vous souhaitez enregistrer le DataFrame
nom_fichier_csv = 'Df_SOL_W1.csv'

# Utilisez la méthode to_csv() pour sauvegarder le DataFrame au format CSV
df.to_csv(nom_fichier_csv, index=False)